<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/pytorch_sensitivities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sensitivity of Option Price to Implied Volatility Surface


The sensitivity of the option price $ C(K, T) $ to the implied volatility surface c \sigma_{\text{implied}}(K, T) $ is computed as:

$
\frac{\partial C(K, T)}{\partial \sigma_{\text{implied}}(K, T)}.
$

This measures how a small change in the implied volatility at a specific strike $ K $ and maturity $ T $ affects the option price.


### Black-Scholes Option Pricing Formula

The Black-Scholes option pricing formula for a European call option is given by:

$
C(K, T) = S_0 \Phi(d_1) - K e^{-rT} \Phi(d_2),
$

where:

$
d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r + \frac{1}{2} \sigma^2\right) T}{\sigma \sqrt{T}}, \quad
d_2 = d_1 - \sigma \sqrt{T}.
$

**Parameters:**
- $ S_0 $: Spot price of the underlying asset,
- $ K $: Strike price of the option,
- $ T $: Time to maturity,
- $ r $: Risk-free rate,
- $ \sigma $: Volatility,
- $ \Phi(x) $: Cumulative distribution function (CDF) of the standard normal distribution.


### Volatility Surface Representation

The implied volatility surface is represented as a grid of strike prices $ K $ and maturities $ T $:

$
\sigma_{\text{implied}}(K, T) =
\begin{bmatrix}
\sigma_{1,1} & \sigma_{1,2} & \cdots & \sigma_{1,n} \\
\sigma_{2,1} & \sigma_{2,2} & \cdots & \sigma_{2,n} \\
\vdots       & \vdots       & \ddots & \vdots       \\
\sigma_{m,1} & \sigma_{m,2} & \cdots & \sigma_{m,n} \\
\end{bmatrix}.
$


### Sensitivity Calculation

The sensitivity of the option price to the implied volatility surface is calculated by summing the option prices over the grid and taking the gradient with respect to the volatility surface:

$
\frac{\partial C}{\partial \sigma_{\text{implied}}} =
\begin{bmatrix}
\frac{\partial C_{1,1}}{\partial \sigma_{1,1}} & \frac{\partial C_{1,2}}{\partial \sigma_{1,2}} & \cdots & \frac{\partial C_{1,n}}{\partial \sigma_{1,n}} \\
\frac{\partial C_{2,1}}{\partial \sigma_{2,1}} & \frac{\partial C_{2,2}}{\partial \sigma_{2,2}} & \cdots & \frac{\partial C_{2,n}}{\partial \sigma_{2,n}} \\
\vdots                                        & \vdots                                        & \ddots & \vdots                                        \\
\frac{\partial C_{m,1}}{\partial \sigma_{m,1}} & \frac{\partial C_{m,2}}{\partial \sigma_{m,2}} & \cdots & \frac{\partial C_{m,n}}{\partial \sigma_{m,n}} \\
\end{bmatrix}.
$

Using PyTorch, this is implemented as:
```python
torch.autograd.grad(option_prices.sum(), implied_vol_surface)



#### **6. Markdown Cell: Output Interpretation**
### Output Interpretation

The output of the sensitivity calculation is a matrix of the same size as the implied volatility surface. Each entry represents:

$
\text{sensitivity}_{ij} = \frac{\partial C(K_i, T_j)}{\partial \sigma_{\text{implied}}(K_i, T_j)}.
$

This matrix quantifies the effect of changes in implied volatility on option prices across the entire surface.


### Applications

The sensitivity of option prices to the volatility surface is useful for:

- Quantifying the impact of changes in implied volatility on option prices.
- Developing risk management and hedging strategies.
- Calibrating local volatility models for more accurate pricing.


In [ ]:
import torch

# Risk-free rate
r = 0.03  # Example: 3%

# Define maturities (T) and strikes (K)
T = torch.tensor([0.1, 0.5, 1.0, 2.0], requires_grad=True)  # Maturities
K = torch.tensor([50, 60, 70, 80, 90, 100, 110, 120, 130],
                 dtype=torch.float32, requires_grad=True)

# Implied volatility surface
implied_vol_surface = torch.tensor(
    [
        [0.20, 0.19, 0.18, 0.17, 0.16, 0.15, 0.16, 0.17, 0.18],
        [0.21, 0.20, 0.19, 0.18, 0.17, 0.16, 0.17, 0.18, 0.19],
        [0.22, 0.21, 0.20, 0.19, 0.18, 0.17, 0.18, 0.19, 0.20],
        [0.23, 0.22, 0.21, 0.20, 0.19, 0.18, 0.19, 0.20, 0.21],
    ],
    requires_grad=True,
)

# Spot price
S0 = 100  # Spot price

# Black-Scholes option pricing function
def option_price(S0, K, T, sigma, r):
    d1 = (torch.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)
    N = torch.distributions.Normal(0, 1)
    return S0 * N.cdf(d1) - K * torch.exp(-r * T) * N.cdf(d2)

# Calculate option prices
option_prices = option_price(S0, K.view(1, -1), T.view(-1, 1), implied_vol_surface, r)

# Sensitivity of option price to volatility surface
sensitivity_to_volatility_surface = torch.autograd.grad(
    outputs=option_prices.sum(),
    inputs=implied_vol_surface,
    create_graph=True,
)[0]

# Print the sensitivity matrix
print("Sensitivity of Option Price to Volatility Surface:")
print(sensitivity_to_volatility_surface)


In [ ]:
delta = 1e-5  # Small perturbation

In [ ]:
# Function to calculate the option price
def compute_option_prices(implied_vol_surface):
    return option_price(S0, K.view(1, -1), T.view(-1, 1), implied_vol_surface, r)

# Central finite difference approximation
finite_diff_sensitivity = torch.zeros_like(implied_vol_surface)

for i in range(implied_vol_surface.shape[0]):  # Iterate over rows (maturities)
    for j in range(implied_vol_surface.shape[1]):  # Iterate over columns (strikes)
        # Create perturbed volatility surfaces
        vol_plus = implied_vol_surface.clone()
        vol_minus = implied_vol_surface.clone()

        # Apply perturbations
        vol_plus[i, j] += delta
        vol_minus[i, j] -= delta

        # Compute option prices for perturbed surfaces
        C_plus = compute_option_prices(vol_plus)
        C_minus = compute_option_prices(vol_minus)

        # Compute finite difference
        finite_diff_sensitivity[i, j] = (C_plus.sum() - C_minus.sum()) / (2 * delta)


In [ ]:
# Autograd sensitivity
autograd_sensitivity = torch.autograd.grad(option_prices.sum(),
                                           implied_vol_surface, retain_graph=True)[0]

# Print the results
print("Finite Difference Sensitivity:")
print(finite_diff_sensitivity)

print("Autograd Sensitivity:")
print(autograd_sensitivity)


In [ ]:
# Compute the relative error
relative_error = torch.norm(finite_diff_sensitivity - autograd_sensitivity) / torch.norm(autograd_sensitivity)
print(f"Relative Error: {relative_error.item()}")
